In [1]:
from database import database
from matplotlib import pyplot as plt
import sklearn.feature_extraction.image
from sklearn.feature_extraction.image import extract_patches
import tables
from SRCNN import run

In [2]:
args = {}
args['filedir'] = 'train_data'
args['pattern'] = '*.bmp'
args['database_name'] = 'mock'
args['export_dir'] = 'export'
args['patch_shape'] = (64,64,3)
args['stride_size'] = 32
data_set = database(**args)

In [3]:
run(data_set, num_epoch = 50, batch_size = 20)

 10%|▉         | 8/81 [00:00<00:00, 78.14it/s]

Preparing data...


100%|██████████| 10/10 [00:00<00:00, 77.77it/s]


Instructions for updating:
Use `tf.global_variables_initializer` instead.
EPOCH: 0


InvalidArgumentError: Incompatible shapes: [12,64,64,3] vs. [20,64,64,3]
	 [[node gradients/sub_grad/BroadcastGradientArgs (defined at /Users/khadinhluong/SRCNN/SRCNN.py:49)  = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gradients/sub_grad/Shape, gradients/sub_grad/Shape_1)]]

Caused by op 'gradients/sub_grad/BroadcastGradientArgs', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File